In [ ]:
!pip -q install agent-framework --pre openai


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "your-key-here"
os.environ["OPENAI_CHAT_MODEL_ID"] = "gpt-4o-mini"  # change to any model you have


In [ ]:
#!pip -q install agent-framework-core

import os
from typing import Any
from contextlib import AsyncExitStack
from IPython.display import clear_output

from agent_framework import (
    ChatAgent,
    Executor,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,
    AgentRunUpdateEvent,
    MCPStreamableHTTPTool,  # <-- HTTP/SSE MCP
    handler,
)
from agent_framework.openai import OpenAIChatClient

# ---------- CONFIG ----------
#os.environ.setdefault("OPENAI_API_KEY", "<YOUR_OPENAI_API_KEY>")
#os.environ.setdefault("OPENAI_CHAT_MODEL_ID", "gpt-4o-mini")

MCP_URL = "Your_deployed_URL"  # your Railway server or any other
MCP_HEADERS = {}  # e.g., {"Authorization": "Bearer <token>"} if your server needs it

# ---------- EXECUTORS ----------
class Dispatcher(Executor):
    """Fan-out source."""
    @handler
    async def handle(self, request: str, ctx: WorkflowContext[str]):
        if not isinstance(request, str) or not request.strip():
            raise RuntimeError("Input must be a non-empty string.")
        await ctx.send_message(request)

class AggregatePlaylist(Executor):
    """Collects agent results and yields a final playlist report; also stores sections for transcript."""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.last_sections = {}

    def _as_text(self, r):
        try:
            ar = getattr(r, "agent_run_response", None)
            if ar is not None and getattr(ar, "text", None):
                return ar.text
        except Exception:
            pass
        return str(r)

    @handler
    async def handle(self, results: list[Any], ctx: WorkflowContext[None, str]):
        sections = {}
        for r in results:
            sections[getattr(r, "executor_id", "unknown")] = self._as_text(r)
        self.last_sections = sections

        discovery = sections.get("Discovery", "")
        trends    = sections.get("Trends", "")
        recos     = sections.get("Recommendations", "")

        final = (
            "# Movie Night: Curated Playlist\n\n"
            "## Discovery (query-based)\n"
            f"{discovery}\n\n"
            "## Trends (week)\n"
            f"{trends}\n\n"
            "## Recommendations (seed-based)\n"
            f"{recos}\n\n"
            "## Final Playlist (editor’s cut)\n"
            "- Pick 2 from Trends and 3 from Recommendations.\n"
            "- Ensure variety of year/genre and family-friendly options where asked.\n"
            "- Prefer entries repeated across sections.\n"
        )
        await ctx.yield_output(final)

# ---------- AGENTS ----------
def make_discovery_agent(tmdb_tool) -> ChatAgent:
    # Uses MCP tools: search_movies
    return ChatAgent(
        chat_client=OpenAIChatClient(),
        name="Discovery",
        instructions=(
            "You are Discovery. Use the MCP tool 'tmdb' strictly for data retrieval.\n"
            "1) Call `search_movies` with {query: <user theme/keywords>}.\n"
            "2) Return a short, clean list (5 items max): 'Title (Year) — TMDB id: <id> — Rating — 1-line overview'.\n"
            "No extra prose. If nothing found, say 'No results'."
        ),
        tools=[tmdb_tool],
    )

def make_trends_agent(tmdb_tool) -> ChatAgent:
    # Uses MCP tools: get_trending (timeWindow = 'week')
    return ChatAgent(
        chat_client=OpenAIChatClient(),
        name="Trends",
        instructions=(
            "You are Trends. Use the MCP tool 'tmdb'.\n"
            "Call `get_trending` with {timeWindow:'week'}.\n"
            "Return a list (top 5): 'Title (Year) — TMDB id — Rating — 1-line why it’s trending'."
        ),
        tools=[tmdb_tool],
    )

def make_recos_agent(tmdb_tool) -> ChatAgent:
    # Uses MCP tools: search_movies (to find a seed id if needed) + get_recommendations
    return ChatAgent(
        chat_client=OpenAIChatClient(),
        name="Recommendations",
        instructions=(
            "You are Recommendations. Use the MCP tool 'tmdb'.\n"
            "If user provided a seed title, first call `search_movies` to get its TMDB id. "
            "Then call `get_recommendations` with {movieId: <id>}.\n"
            "Return up to 5 recs: 'Title (Year) — TMDB id — Rating — 1-line reason'."
        ),
        tools=[tmdb_tool],
    )

# ---------- ORCHESTRATOR (tidy streaming) ----------
async def run_orchestrator(user_request: str) -> None:
    dispatcher = Dispatcher(id="dispatcher")
    aggregator = AggregatePlaylist(id="aggregate")

    async with AsyncExitStack() as stack:
        # Connect to your remote MCP server over HTTP/SSE
        try:
            tmdb_tool = await stack.enter_async_context(
                MCPStreamableHTTPTool(
                    name="tmdb",
                    url=MCP_URL,
                    #headers=MCP_HEADERS or None,
                )
            )
            mcp_status = f"✅ Connected MCP tool @ {MCP_URL}"
        except Exception as e:
            # If your server requires auth or is unreachable, show a clear message
            mcp_status = f"❌ MCP connection failed: {e}"
            tmdb_tool = None

        discovery = make_discovery_agent(tmdb_tool) if tmdb_tool else ChatAgent(
            chat_client=OpenAIChatClient(), name="Discovery",
            instructions="TMDB tool unavailable. Explain that the MCP server could not be reached."
        )
        trends = make_trends_agent(tmdb_tool) if tmdb_tool else ChatAgent(
            chat_client=OpenAIChatClient(), name="Trends",
            instructions="TMDB tool unavailable. Explain that the MCP server could not be reached."
        )
        recos = make_recos_agent(tmdb_tool) if tmdb_tool else ChatAgent(
            chat_client=OpenAIChatClient(), name="Recommendations",
            instructions="TMDB tool unavailable. Explain that the MCP server could not be reached."
        )

        workflow = (
            WorkflowBuilder()
            .set_start_executor(dispatcher)
            .add_fan_out_edges(dispatcher, [discovery, trends, recos])
            .add_fan_in_edges([discovery, trends, recos], aggregator)
            .build()
        )

        # Neat transcript
        buffers = {"Discovery": "", "Trends": "", "Recommendations": ""}

        def render(final_report: str | None = None):
            clear_output(wait=True)
            print("=== Enterprise Orchestrator: TMDB over HTTP (MCP) ===")
            print(mcp_status, "\n")
            print("--- Live transcript ---\n")
            for name in ["Discovery", "Trends", "Recommendations"]:
                print(f"### {name}\n{buffers[name]}\n")
            if final_report is not None:
                print("\n===== FINAL REPORT =====\n")
                print(final_report)

        render()
        got_updates = False
        async for event in workflow.run_stream(user_request):
            if isinstance(event, AgentRunUpdateEvent):
                eid = event.executor_id
                text = getattr(event, "data", "")
                if isinstance(text, str) and text:
                    buffers.setdefault(eid, "")
                    buffers[eid] += text
                    got_updates = True
                    render()
            elif isinstance(event, WorkflowOutputEvent):
                if not got_updates and getattr(aggregator, "last_sections", None):
                    for k, v in aggregator.last_sections.items():
                        buffers[k] = v
                render(final_report=event.data)
                break

# ---------- DEMO REQUEST ----------
# Feel free to change the theme/seed title:
demo_request = (
    "Plan a family-friendly weekend movie night around 'Inception' as a seed title. "
    "Give 2 trending-week picks and 3 recommendations similar to the seed. "
    "Keep each line concise with Title (Year), TMDB id, rating, and a 1-line reason."
)

import asyncio
await run_orchestrator(demo_request)


=== Enterprise Orchestrator: TMDB over HTTP (MCP) ===
❌ MCP connection failed: Failed to enter context manager. 

--- Live transcript ---

### Discovery
**Seed Title:** Inception (2010)

**Trending Picks:**

1. **Dune (2021)**  
   TMDB ID: 528085  
   Rating: 8.1  
   A visually stunning adaptation of a sci-fi classic that offers exciting adventure and rich storytelling.

2. **Spider-Man: No Way Home (2021)**  
   TMDB ID: 581386  
   Rating: 8.0  
   A thrilling multiverse adventure filled with heart and humor, perfect for the whole family.

**Similar Recommendations:**

1. **Interstellar (2014)**  
   TMDB ID: 157336  
   Rating: 8.6  
   A mind-bending journey through space and time with emotional depth and stunning visuals.

2. **The Matrix (1999)**  
   TMDB ID: 603  
   Rating: 8.7  
   A revolutionary sci-fi journey that explores reality and choice in an action-packed narrative.

3. **Shutter Island (2010)**  
   TMDB ID: 76341  
   Rating: 8.2  
   A gripping psychological thr